In [30]:
import re
import pandas as pd
import plotly.express as px

from glob import glob

In [31]:
pt_time = re.compile(r"\[epoch 2\] elapsed time\: (\d+\.\d+) sec")
pt_np = re.compile(r"np(\d+)")

In [32]:
standards = {}

for log in glob("./**/np8-bs1/torch.log"):
    with open(log, "r") as f:
        lines = f.readlines()

    for line in lines:
        sh_time = pt_time.search(line)
        if sh_time:
            elapsed_time = int(float(sh_time.group(1)))

    model = log.split("/")[1]
    standards[model] = elapsed_time

In [36]:
logs = sorted(glob("./**/**/torch.log"))
rows = []

for log in logs:
    with open(log, "r") as f:
        lines = f.readlines()

    for line in lines:
        sh_time = pt_time.search(line)
        if sh_time:
            elapsed_time = int(float(sh_time.group(1)))

    model = log.split("/")[1]
    np = int(pt_np.search(log).group(1))

    if np == 8:
        efficiency = 100
    else:
        efficiency = int(standards[model] / (elapsed_time * (np // 8)) * 100)

    rows.append(
        {
            "model": model,
            "nodes": np // 8,
            "gpus": np,
            "elapsed_time": elapsed_time,
            "efficiency": efficiency,
        }
    )

In [37]:
df = pd.DataFrame(rows)
df["model"] = pd.Categorical(df["model"], categories=("Llama-2-7b-chat-hf", "Llama-2-13b-chat-hf", "Llama-2-70b-chat-hf"))
df.sort_values(by=["model", "nodes"], ascending=[True, True], inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv("data_ml.csv", index=False)
df

,model,nodes,gpus,elapsed_time,efficiency
0,Llama-2-7b-chat-hf,1,8,14712,100
1,Llama-2-7b-chat-hf,2,16,2753,267
2,Llama-2-7b-chat-hf,4,32,1079,340
3,Llama-2-7b-chat-hf,8,64,306,600
4,Llama-2-7b-chat-hf,16,128,140,656
5,Llama-2-7b-chat-hf,24,192,126,486
6,Llama-2-7b-chat-hf,32,256,103,446
7,Llama-2-7b-chat-hf,40,320,100,367
8,Llama-2-7b-chat-hf,48,384,94,326
9,Llama-2-13b-chat-hf,1,8,13737,100
